<a href="https://colab.research.google.com/github/Tranvanhien141625/BDS/blob/main/16_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U -q "langchain" "transformers==4.31.0" "datasets==2.13.0" "peft==0.4.0" "accelerate==0.21.0" "bitsandbytes==0.40.2" "trl==0.4.7" "safetensors>=0.3.1"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.4/802.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.9/218.9 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install -q -U faiss-cpu tiktoken sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.


In [ ]:
!pip install ctransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 17.6 MB/s eta 0:00:00


In [ ]:
from langchain_community.llms import CTransformers
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.vectorstores import FAISS

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_community.document_loaders import TextLoader


In [ ]:
documents = TextLoader('/content/BDS2.txt').load()
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size = 400, # the character length of the chunk
    chunk_overlap = 30, # the character length of the overlap between chunks
    length_function = len, # the length function - in this case, character length (aka the python len() fn.)
)

chunks = text_splitter.split_documents(documents)


In [ ]:
from langchain.embeddings import CacheBackedEmbeddings, HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
store = LocalFileStore("./cache/")
embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'

core_embeddings_model = HuggingFaceEmbeddings(
    model_name=embed_model_id
)

embedder = CacheBackedEmbeddings.from_bytes_store(
    core_embeddings_model, store, namespace=embed_model_id
)

db = FAISS.from_documents(chunks,embedder)

In [ ]:

# # Khai bao bien
# pdf_data_path = "data"
# vector_db_path = "vectorstores/db_faiss"

# # Ham 1. Tao ra vector DB tu 1 doan text
# raw_text = """Cho thuê căn hộ dịch vụ Mai Động, Phường Mai Động, Quận Hoàng Mai, giá 4tr, diện tích 70m2. Thông tin phòng GIÁ PHÒNG 4,000,000 đồng. ĐẶT CỌC 4,000,000 đồng. ĐIỆN Miễn phí NƯỚC Miễn phí, WIFI Miễn phí. TRẠNG THÁI Còn phòng. ĐIẠ CHỈ 104C39 tập thể mai động Mai động, Phường Mai Động, Quận Hoàng Mai, Hà Nội. Tiện ích Máy lạnh WC riêng Chổ để xe Wifi Tự do Không chung chủ . Bảo vệ Nấu ăn Máy nước nóng. Lưu ý Tiện ích Không cửa sổ. Sức chứa Rộng 6. Mô tả thêm Điện nước tự thanh toán qua momo giá nhà nước! Mùa hè nhà siêu mát, rộng rãi, thoải mái sửa chữa và decor
# """
#     # Chia nho van ban
# text_splitter = CharacterTextSplitter(
# separator="\n",
# chunk_size=512,
# chunk_overlap=50,
# length_function=len

# )

# chunks = text_splitter.split_text(raw_text)

#     # Embeding
# embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'

# embeddings_model = HuggingFaceEmbeddings(model_name=embed_model_id)

#     # Dua vao Faiss Vector DB
# db = FAISS.from_texts(texts=chunks, embeddings_model)




In [ ]:
model_file="bigscience/bloomz-7b1-mt"

In [ ]:
def load_llm(model_file):
    llm = CTransformers(
        model=model_file,
        model_type="llama",
        max_new_tokens=1024,
        temperature=0.01
    )
    return llm

In [ ]:
# Tao prompt template
def creat_prompt(template):
    prompt = PromptTemplate(template = template, input_variables=["context", "question"])
    return prompt


In [ ]:
def create_qa_chain(prompt, llm, db):
    llm_chain = RetrievalQA.from_chain_type(
        llm = llm,
        chain_type= "stuff",
        retriever = db.as_retriever(search_kwargs = {"k":3}, max_tokens_limit=1024),
        return_source_documents = False,
        chain_type_kwargs= {'prompt': prompt}

    )
    return llm_chain


In [ ]:
# # Read tu VectorDB
# def read_vectors_db():
#     # Embeding
#     embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'

#     embeddings_model = HuggingFaceEmbeddings(
#     model_name=embed_model_id
#       )
#     db = FAISS.load_local(vector_db_path, embedding_model)
#     return db


In [ ]:
# Tao prompt template
def creat_prompt(template):
    prompt = PromptTemplate(template = template, input_variables=["context", "question"])
    return prompt



In [ ]:
!pip install huggingface-hub -q

In [ ]:
import os

os.system("sudo apt install git")
!sudo apt install git


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.10).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [ ]:
!git clone https://huggingface.co/vilm/vinallama-7b-chat-GGUF

fatal: destination path 'vinallama-7b-chat-GGUF' already exists and is not an empty directory.


In [ ]:

# Bat dau thu nghiem
#db = read_vectors_db()
llm = load_llm(model_file)

#Tao Prompt
template = """<|im_start|>system\nSử dụng thông tin sau đây để trả lời câu hỏi. Nếu bạn không biết câu trả lời, hãy nói không biết, đừng cố tạo ra câu trả lời\n
    {context}<|im_end|>\n<|im_start|>user\n{question}<|im_end|>\n<|im_start|>assistant"""
prompt = creat_prompt(template)

llm_chain  =create_qa_chain(prompt, llm, db)

# Chay cai chain
question = "Điện nước thanh toán thế nào?"
response = llm_chain.invoke({"query": question})
print(response)

{'query': 'Điện nước thanh toán thế nào?', 'result': '\nĐiện nước được tính riêng. Bạn có thể chọn giữa phương thức cấp hoặc công suất tùy theo nhu cầu của mình. Thông tin cụ thể về các phương thức và giá cả sẽ được cung cấp trong hợp đồng thuê nhà bạn sắp ký kết. Hãy đảm bảo đọc kỹ điều khoản trước khi xác nhận thỏa thuận.'}
